In [1]:
import pandas as pd
import json
%load_ext examples

In [2]:
dataset = pd.read_csv('Examples/Electricity_Production_By_Source.csv')
agg = dataset.groupby(['Entity']).max().reset_index()
agg = dataset.groupby(['Entity']).max().reset_index()

out = pd.DataFrame();

agg

,Entity,Code,Year,Electricity from coal (TWh),Electricity from gas (TWh),Electricity from hydro (TWh),Electricity from other renewables (TWh),Electricity from solar (TWh),Electricity from oil (TWh),Electricity from wind (TWh),Electricity from nuclear (TWh)
0,Afghanistan,AFG,2019,0.000000,0.344000,1.050000,0.000000,0.040000,0.000000,0.000100,0.000000
1,Africa,NaN,2019,262.470414,340.472820,132.734791,8.184247,19.377978,92.816685,17.508052,15.913439
2,Albania,ALB,2019,0.000000,0.000000,8.466480,0.000000,0.022000,0.163560,0.000000,0.000000
3,Algeria,DZA,2019,0.000000,73.615787,0.646000,0.000000,0.615000,0.000000,0.029000,0.000000
4,American Samoa,ASM,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.187000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
230,Western Sahara,ESH,2019,0.000000,0.000000,0.000000,0.000000,0.000000,0.090000,0.000000,0.000000
231,World,OWID_WRL,2020,9345.340997,5943.112329,4355.041636,702.887552,844.385951,1364.578206,1590.189440,2720.674471
232,Yemen,YEM,2019,0.000000,3.058733,0.000000,0.000000,0.500000,6.339360,0.000000,0.000000
233,Zambia,ZMB,2019,1.998501,0.000000,13.902000,0.080073,0.002432,1.003232,0.000000,0.000000


In [4]:
%scatter_plt agg ?out

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
%show_table ?out

<IPython.core.display.Javascript object>

In [45]:
%scatter_plt out ?out

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
%get_filter fltr

<IPython.core.display.Javascript object>

In [8]:
agg = out

In [22]:
fltr = json.loads(fltr)
agg = agg[~agg.index.isin(fltr)]